**Disclaimer:** The distributed notebook only serves for demonstration purposes. Jupyter notebooks are not recommended for preCICE development or deployment. Feel free to convert this notebook to a plain python script for improved error messages using `jupyter nbconvert --to python Solver*.ipynb`

In [1]:
def dummyFluidSolver(x, y, dt):
    return x + dt * (-y)

In [2]:
import precice
#precice_config = "precice-config-both-init.xml"
#precice_config = "precice-config-init-serial.xml"
precice_config = "precice-config-experimental-both-init.xml"
interface = precice.Interface("Fluid", precice_config, 0,1)
meshId = interface.get_mesh_id("Fluid-Mesh")
vertexId = interface.set_mesh_vertex(meshId, [1,2,3])
writeDataId = interface.get_data_id("Force", meshId)
readDataId = interface.get_data_id("Displacement", meshId)

(0) 17:58:44 [impl::SolverInterfaceImpl]:169 in configure: This is preCICE version 2.3.0
(0) 17:58:44 [impl::SolverInterfaceImpl]:170 in configure: Revision info: v2.3.0-120-g5b868a24
(0) 17:58:44 [impl::SolverInterfaceImpl]:172 in configure: Configuration: Debug
(0) 17:58:44 [impl::SolverInterfaceImpl]:176 in configure: Configuring preCICE with configuration "precice-config-init-serial.xml"
(0) 17:58:44 [impl::SolverInterfaceImpl]:177 in configure: I am participant "Fluid"


In [3]:
precice_dt = interface.initialize()

(0) 17:58:44 [impl::SolverInterfaceImpl]:250 in initialize: Setting up master communication to coupling partner/s
(0) 17:58:48 [impl::SolverInterfaceImpl]:264 in initialize: Masters are connected
(0) 17:58:48 [impl::SolverInterfaceImpl]:268 in initialize: Setting up preliminary slaves communication to coupling partner/s
(0) 17:58:48 [partition::ProvidedPartition]:121 in prepare: Prepare partition for mesh Fluid-Mesh
(0) 17:58:48 [partition::ProvidedPartition]:88 in communicate: Gather mesh Fluid-Mesh
(0) 17:58:48 [partition::ProvidedPartition]:105 in communicate: Send global mesh Fluid-Mesh
(0) 17:58:48 [impl::SolverInterfaceImpl]:276 in initialize: Setting up slaves communication to coupling partner/s
(0) 17:58:48 [impl::SolverInterfaceImpl]:282 in initialize: Slaves are connected
(0) 17:58:48 [impl::SolverInterfaceImpl]:316 in initialize: iteration: 1 of 2, time-window: 1 of 2, time: 0, time-window-size: 1, max-timestep-length: 1, ongoing: yes, time-window-complete: no, write-iterati

### Initialize data
optional, if `initialize = "false"` or not given

In [4]:
initForce = 1
writeData = initForce
interface.write_scalar_data(writeDataId, vertexId, writeData)

In [5]:
interface.mark_action_fulfilled(precice.action_write_initial_data())

In [6]:
interface.initialize_data()

### First window, first iteration
require to write checkpoint, when entering window:

In [7]:
interface.mark_action_fulfilled(precice.action_write_iteration_checkpoint())

In [8]:
readData = interface.read_scalar_data(readDataId, vertexId)
print("readData = {}".format(readData))

readData = 2.0


In [9]:
writeData = dummyFluidSolver(writeData, readData, precice_dt)

In [10]:
print("writeData = {}".format(writeData))
interface.write_scalar_data(writeDataId, vertexId, writeData)

writeData = -1.0


In [11]:
precice_dt = interface.advance(precice_dt)
print("precice_dt = {}".format(precice_dt))

precice_dt = 1.0
(0) 18:02:33 [impl::SolverInterfaceImpl]:456 in advance: iteration: 2 of 2, time-window: 1 of 2, time: 0, time-window-size: 1, max-timestep-length: 1, ongoing: yes, time-window-complete: no, read-iteration-checkpoint 


In [ ]:
interface.is_time_window_complete()

### First window, second iteration
require to read checkpoint, when repeating window:

In [ ]:
interface.mark_action_fulfilled(precice.action_read_iteration_checkpoint())

In [ ]:
readData = interface.read_scalar_data(readDataId, vertexId)
print("readData = {}".format(readData))

In [ ]:
writeData = dummyFluidSolver(writeData, readData, precice_dt)

In [ ]:
print("writeData = {}".format(writeData))
interface.write_scalar_data(writeDataId, vertexId, writeData)

In [ ]:
precice_dt = interface.advance(precice_dt)
print("precice_dt = {}".format(precice_dt))

In [ ]:
interface.is_time_window_complete()

### Second window:
... same procedure as before.

In [ ]:
window = 2
iteration = 1

while interface.is_coupling_ongoing():
    if interface.is_action_required(precice.action_write_iteration_checkpoint()):
        interface.mark_action_fulfilled(precice.action_write_iteration_checkpoint())
        
    readData = interface.read_scalar_data(readDataId, vertexId)
    print("readData = {}".format(readData))
    
    writeData = dummyFluidSolver(writeData, readData, precice_dt)
    
    print("writeData = {}".format(writeData))
    interface.write_scalar_data(writeDataId, vertexId, writeData)
    
    precice_dt = interface.advance(precice_dt)
    print("precice_dt = {}".format(precice_dt))
    
    if interface.is_time_window_complete():
        window += 1
        iteration = 1
    if interface.is_action_required(precice.action_read_iteration_checkpoint()):
        iteration += 1
        
    if interface.is_coupling_ongoing():
        print("window = {}".format(window))
        print("iteration = {}".format(iteration))
        
    if interface.is_action_required(precice.action_read_iteration_checkpoint()):
        interface.mark_action_fulfilled(precice.action_read_iteration_checkpoint())
        
print("Done!")
interface.finalize()